In [1]:
#!pip install prefect[dask] dask-jobqueue


In [2]:
#!prefect config set PREFECT_API_URL="https://ard-modeling-service.slac.stanford.edu/api" 


In [3]:
#!python -m pip install jupyter-server-proxy

In [15]:

from dask.distributed import Client
from dask_jobqueue import SLURMCluster

# Simplify SLURMCluster parameters to basic working configuration

#SBATCH --qos=debug
#SBATCH --time=5
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=128
#SBATCH --constraint=cpu


cluster_kwargs = {
    "cores": 1,  
    "memory": "10GB",  # Correct spacing
    "shebang": "#!/bin/bash",
    #"account": "sanjeevc",
    "walltime": "00:10:00",
    "job_script_prologue": ["source ~/.bashrc"],
    # Only basic directives, add more as needed and test each step
    "job_extra_directives": ["-A m669","-q regular","--constraint=cpu"],
        "asynchronous": True
}
cluster = SLURMCluster(**cluster_kwargs)

# Modify the network interface to 'hsn0'
#cluster.scheduler_options = {"interface": "hsn0"}

print(cluster.job_script())


slurm_jobs = 10
cluster.scale(jobs=slurm_jobs)
client = Client(cluster)
client

#!/bin/bash

#SBATCH -J dask-worker
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=10G
#SBATCH -t 00:10:00
#SBATCH -A m669
#SBATCH -q regular
#SBATCH --constraint=cpu
source ~/.bashrc
/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/bin/python -m distributed.cli.dask_worker tcp://<insert-scheduler-address-here>:8786 --name dummy-name --nthreads 1 --memory-limit 9.31GiB --nanny --death-timeout 60



Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster


/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38991 instead
  warnings.warn(


In [ ]:
!dask-scheduler


/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/lib/python3.11/site-packages/distributed/cli/dask_scheduler.py:142: FutureWarning: dask-scheduler is deprecated and will be removed in a future release; use `dask scheduler` instead
  warnings.warn(
2024-09-10 19:01:02,880 - distributed.scheduler - INFO - -----------------------------------------------
/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42325 instead
  warnings.warn(
2024-09-10 19:01:03,576 - distributed.scheduler - INFO - State start
2024-09-10 19:01:03,585 - distributed.scheduler - INFO - -----------------------------------------------
2024-09-10 19:01:03,586 - distributed.scheduler - INFO -   Scheduler at:   tcp://128.55.64.38:8786
2024-09-10 19:01:03,587 - distributed.scheduler - INFO -   dashboard

In [6]:
#!prefect config set PREFECT_API_URL="https://ard-modeling-service.slac.stanford.edu/api" 

In [7]:
#!prefect config set PREFECT_API_URL="http://localhost:8000/api"


/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43569 instead
  warnings.warn(


In [8]:
%time


CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 5.01 µs


In [ ]:
client

In [10]:
print(client.dashboard_link)

In [11]:
from prefect import flow, get_run_logger, task
from prefect_dask.task_runners import DaskTaskRunner


@task
def say_hello(name: str) -> None:
    # logs not currently working see https://github.com/PrefectHQ/prefect/issues/5850
    logger = get_run_logger()
    logger.info(f"hello {name}")


@task
def say_goodbye(name: str) -> None:
    logger = get_run_logger()
    logger.info(f"goodbye {name}")


# TODO: can the task runner be parameterised so we don't duplicate the flow with dask_kubes_flow?
# see https://github.com/PrefectHQ/prefect/issues/5560


# creates a LocalCluster https://docs.dask.org/en/stable/deploying-python.html#localcluster
@flow(task_runner=DaskTaskRunner(cluster=cluster))
def dask(names: list[str]) -> None:
    for name in names:
        # tasks must be submitted to run on dask
        # if called without .submit() they are still tracked but
        # run immediately and locally rather than async on dask
        say_hello.submit(name)
        say_goodbye.submit(name)


if __name__ == "__main__":
    dask(["arthur", "trillian", "ford", "marvin"])

19:00:09.829 | INFO    | prefect.engine - Created flow run 'smoky-dolphin' for flow 'dask'

19:00:09.831 | INFO    | Flow run 'smoky-dolphin' - View at https://ard-modeling-service.slac.stanford.edu/flow-runs/flow-run/f269ea03-49b0-4b91-a5d4-1ea2be1c790d

19:00:09.833 | INFO    | prefect.task_runner.dask - Creating a new Dask cluster with `distributed.deploy.local.LocalCluster`

/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46009 instead
  warnings.warn(


19:00:14.359 | INFO    | prefect.task_runner.dask - The Dask dashboard is available at http://127.0.0.1:46009/status

...

19:00:14.518 | INFO    | Flow run 'smoky-dolphin' - Submitted task run 'say_hello-2' for execution.

19:00:14.524 | INFO    | Flow run 'smoky-dolphin' - Created task run 'say_hello-3' for task 'say_hello'

19:00:14.528 | INFO    | Flow run 'smoky-dolphin' - Submitted task run 'say_hello-3' for execution.

19:00:16.954 | INFO    | Task run 'say_goodbye-0' - goodbye arthur
19:00:16.982 | INFO    | Task run 'say_goodbye-0' - Finished in state Completed()
19:00:17.105 | INFO    | Task run 'say_goodbye-1' - goodbye trillian
19:00:17.121 | INFO    | Task run 'say_hello-0' - hello arthur
19:00:17.122 | INFO    | Task run 'say_goodbye-3' - goodbye marvin
19:00:17.158 | INFO    | Task run 'say_hello-3' - hello marvin
19:00:17.171 | INFO    | Task run 'say_goodbye-1' - Finished in state Completed()
19:00:17.179 | INFO    | Task run 'say_hello-1' - hello trillian
19:00:17.183 | INFO    | Task run 'say_hello-0' - Finished in state Completed()
19:00:17.183 | INFO    | Task run 'say_hello-2' - hello ford
19:00:17.214 | INFO    | Task run 'say_hello-3' - Finished in state Completed()
19:00:17.241 | INFO    | Task run 'say_hello-2' - Finished in state Completed()
19:00:17.250 | INFO    | Task run 'say_hello-1' - Finished in state Completed()
19:00:17.252 | INFO    | Task run 'say_goodbye-3' - Finished 

19:00:18.578 | INFO    | Flow run 'smoky-dolphin' - Finished in state Completed('All states completed.')

In [12]:
client.shutdown()


<coroutine object Client._shutdown at 0x7f64ae2925c0>

# Temporary Dask Cluster
Some users may prefer to spin up a Dask cluster (i.e. Slurm job) for each indiviual @flow. This is also possible in Prefect. (Note that the example below will complain about an open port if you have already instantiated a Dask cluster above.)



from prefect import flow, task
from prefect_dask import DaskTaskRunner

@flow(task_runner=DaskTaskRunner(cluster_class=SLURMCluster, cluster_kwargs=cluster_kwargs))
def workflow(a: float, b: float) -> float:
    output1 = add.submit(a, b)
    output2 = mult.submit(output1, b)
    return output2

@task
def add(a: float, b: float) -> float:
    return a + b

@task
def mult(a: float, b: float) -> float:
    return a * b

workflow(1, 2).result()

In [13]:
from dask.distributed import Client, default_client

# Function to safely shutdown the client
def shutdown_all_clients():
    try:
        client = default_client()  # Get the current active client
        client.sync(client.shutdown)  # Synchronously wait for the shutdown
        print("Client successfully shut down.")
    except ValueError:
        print("No active client found.")

shutdown_all_clients()


Client successfully shut down.


/tmp/ipykernel_2043352/4030458115.py:7: RuntimeWarning: coroutine 'Client._shutdown' was never awaited
  client.sync(client.shutdown)  # Synchronously wait for the shutdown


In [14]:
from dask.distributed import Client

# Connect to the cluster running on port 8787
client = Client("tcp://localhost:8787")

# Shutdown the cluster
client.shutdown()

print("Cluster on port 8787 successfully shut down.")



KeyboardInterrupt

